In [1]:
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder, MultiLabelBinarizer
import pandas as pd
import numpy as np
import re
import torch
from gensim.models import Word2Vec
from ast import literal_eval


In [2]:
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 10)
pd.options.display.max_colwidth = 50

In [3]:
df = pd.read_csv('../data/cards_data.csv')
df.head()


,id,name,cmc,mana_cost,colors,legendary,types,sub_types,oracle_text,keywords,power,toughness,textless,rarity,reserved,released_at,booster,usd
0,e882c9f9-bf30-46b6-bedc-379d2c80e5cb,+2 Mace,2.0,{1}{W},['W'],False,['Artifact'],['Equipment'],Equipped creature gets +2/+2.\nEquip {3} ({3}:...,['Equip'],non-creature,non-creature,False,common,False,2021-07-23,True,0.02
1,2a83882c-3e03-4e85-aaac-97fa1d08a772,Aarakocra Sneak,4.0,{3}{U},['U'],False,['Creature'],"['Bird', 'Rogue']",Flying\nWhen Aarakocra Sneak enters the battle...,['Flying'],1,4,False,common,False,2022-06-10,True,0.08
2,c9f1fc97-00c0-492b-a4a3-b179afc2f95d,Abaddon the Despoiler,5.0,{2}{U}{B}{R},"['B', 'R', 'U']",True,['Creature'],"['Astartes', 'Warrior']",Trample\nMark of Chaos Ascendant — During your...,"['Mark of Chaos Ascendant', 'Trample']",5,5,False,mythic,False,2022-10-07,False,2.88
3,4945031e-1158-474c-9e50-1ec817acc767,Abandoned Outpost,0.0,{0},['No Colors'],False,['Land'],['none'],Abandoned Outpost enters the battlefield tappe...,['no keywords'],non-creature,non-creature,False,common,False,2001-10-01,True,0.14
4,23b587f3-5eac-45a7-a048-dd73d2a2b74f,Abandoned Sarcophagus,3.0,{3},['C'],False,['Artifact'],['none'],You may cast spells that have a cycling abilit...,['no keywords'],non-creature,non-creature,False,rare,False,2020-04-17,False,0.09


In [4]:
df[df['name'].str.contains('Kozilek')]


,id,name,cmc,mana_cost,colors,legendary,types,sub_types,oracle_text,keywords,power,toughness,textless,rarity,reserved,released_at,booster,usd
1121,9312fada-fc31-46ac-b6a1-cfffd776856f,Artisan of Kozilek,9.0,{9},['C'],False,['Creature'],['Eldrazi'],"When you cast this spell, you may return targe...",['Annihilator'],10,9,False,uncommon,False,2023-08-04,False,0.43
10227,2a4180ae-1fd9-4185-b4f0-9e8f668a5737,Herald of Kozilek,3.0,{1}{U}{R},['C'],False,['Creature'],"['Eldrazi', 'Drone']",Devoid (This card has no color.)\nColorless sp...,['Devoid'],2,4,False,uncommon,False,2015-10-02,True,0.12
11110,17f38740-20fd-4097-90f8-f0c2c2ff7281,Inquisition of Kozilek,1.0,{B},['B'],False,['Sorcery'],['none'],Target player reveals their hand. You choose a...,['no keywords'],non-creature,non-creature,False,uncommon,False,2022-07-08,True,0.31
12281,d27cf7b7-7982-46bd-a559-7789c0e74bae,"Kozilek, Butcher of Truth",10.0,{10},['C'],True,['Creature'],['Eldrazi'],"When you cast this spell, draw four cards.\nAn...",['Annihilator'],12,12,False,mythic,False,2022-07-08,True,38.41
12282,c550d179-32ec-4ad8-91c2-d79320a21cba,Kozilek's Channeler,5.0,{5},['C'],False,['Creature'],['Eldrazi'],{T}: Add {C}{C}.,['no keywords'],4,4,False,common,False,2015-10-02,True,0.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12286,786c6a0f-3f75-45ff-aae9-5c866be279d0,Kozilek's Sentinel,2.0,{1}{R},['C'],False,['Creature'],"['Eldrazi', 'Drone']",Devoid (This card has no color.)\nWhenever you...,['Devoid'],1,4,False,common,False,2015-10-02,True,0.04
12287,a384cd5b-2c6c-4969-bb62-a017e2fc9794,Kozilek's Shrieker,3.0,{2}{B},['C'],False,['Creature'],"['Eldrazi', 'Drone']",Devoid (This card has no color.)\n{C}: Kozilek...,['Devoid'],3,2,False,common,False,2016-01-22,True,0.04
12288,d80665ae-fcb2-48fd-95be-8768293bcef7,Kozilek's Translator,5.0,{4}{B},['C'],False,['Creature'],"['Eldrazi', 'Drone']",Devoid (This card has no color.)\nPay 1 life: ...,['Devoid'],3,5,False,common,False,2016-01-22,True,0.03
12289,c41554e7-2a07-4cc7-b01b-44deed08e588,"Kozilek, the Great Distortion",10.0,{8}{C}{C},['C'],True,['Creature'],['Eldrazi'],"When you cast this spell, if you have fewer th...",['Menace'],12,12,False,mythic,False,2023-08-04,True,2.16


In [5]:
df['mana_cost'].info()


<class 'pandas.core.series.Series'>
RangeIndex: 26052 entries, 0 to 26051
Series name: mana_cost
Non-Null Count  Dtype 
--------------  ----- 
25678 non-null  object
dtypes: object(1)
memory usage: 203.7+ KB


In [6]:
df['mana_cost'] = df['mana_cost'].astype(str)
df['mana_cost'].head()


0          {1}{W}
1          {3}{U}
2    {2}{U}{B}{R}
3             {0}
4             {3}
Name: mana_cost, dtype: object

In [7]:
def convertmc(mana_cost):
    return list(mana_cost.replace('{', '').replace('}', ''))
df['mana_cost'] =  df['mana_cost'].apply(convertmc)
df['mana_cost'].head()


0          [1, W]
1          [3, U]
2    [2, U, B, R]
3             [0]
4             [3]
Name: mana_cost, dtype: object

In [8]:
df.drop(columns=['id'], inplace=True)
df.head()


,name,cmc,mana_cost,colors,legendary,types,sub_types,oracle_text,keywords,power,toughness,textless,rarity,reserved,released_at,booster,usd
0,+2 Mace,2.0,"[1, W]",['W'],False,['Artifact'],['Equipment'],Equipped creature gets +2/+2.\nEquip {3} ({3}:...,['Equip'],non-creature,non-creature,False,common,False,2021-07-23,True,0.02
1,Aarakocra Sneak,4.0,"[3, U]",['U'],False,['Creature'],"['Bird', 'Rogue']",Flying\nWhen Aarakocra Sneak enters the battle...,['Flying'],1,4,False,common,False,2022-06-10,True,0.08
2,Abaddon the Despoiler,5.0,"[2, U, B, R]","['B', 'R', 'U']",True,['Creature'],"['Astartes', 'Warrior']",Trample\nMark of Chaos Ascendant — During your...,"['Mark of Chaos Ascendant', 'Trample']",5,5,False,mythic,False,2022-10-07,False,2.88
3,Abandoned Outpost,0.0,[0],['No Colors'],False,['Land'],['none'],Abandoned Outpost enters the battlefield tappe...,['no keywords'],non-creature,non-creature,False,common,False,2001-10-01,True,0.14
4,Abandoned Sarcophagus,3.0,[3],['C'],False,['Artifact'],['none'],You may cast spells that have a cycling abilit...,['no keywords'],non-creature,non-creature,False,rare,False,2020-04-17,False,0.09


In [9]:
df['sub_types'].fillna("['none']", inplace=True)
df.head()


,name,cmc,mana_cost,colors,legendary,types,sub_types,oracle_text,keywords,power,toughness,textless,rarity,reserved,released_at,booster,usd
0,+2 Mace,2.0,"[1, W]",['W'],False,['Artifact'],['Equipment'],Equipped creature gets +2/+2.\nEquip {3} ({3}:...,['Equip'],non-creature,non-creature,False,common,False,2021-07-23,True,0.02
1,Aarakocra Sneak,4.0,"[3, U]",['U'],False,['Creature'],"['Bird', 'Rogue']",Flying\nWhen Aarakocra Sneak enters the battle...,['Flying'],1,4,False,common,False,2022-06-10,True,0.08
2,Abaddon the Despoiler,5.0,"[2, U, B, R]","['B', 'R', 'U']",True,['Creature'],"['Astartes', 'Warrior']",Trample\nMark of Chaos Ascendant — During your...,"['Mark of Chaos Ascendant', 'Trample']",5,5,False,mythic,False,2022-10-07,False,2.88
3,Abandoned Outpost,0.0,[0],['No Colors'],False,['Land'],['none'],Abandoned Outpost enters the battlefield tappe...,['no keywords'],non-creature,non-creature,False,common,False,2001-10-01,True,0.14
4,Abandoned Sarcophagus,3.0,[3],['C'],False,['Artifact'],['none'],You may cast spells that have a cycling abilit...,['no keywords'],non-creature,non-creature,False,rare,False,2020-04-17,False,0.09


#### Cards that transform had nan colors, I will remove them for now


In [10]:
df['types'] = df['types'].apply(literal_eval)
df['sub_types'] = df['sub_types'].apply(literal_eval)
df['keywords'] = df['keywords'].apply(literal_eval)
df.head()


,name,cmc,mana_cost,colors,legendary,types,sub_types,oracle_text,keywords,power,toughness,textless,rarity,reserved,released_at,booster,usd
0,+2 Mace,2.0,"[1, W]",['W'],False,[Artifact],[Equipment],Equipped creature gets +2/+2.\nEquip {3} ({3}:...,[Equip],non-creature,non-creature,False,common,False,2021-07-23,True,0.02
1,Aarakocra Sneak,4.0,"[3, U]",['U'],False,[Creature],"[Bird, Rogue]",Flying\nWhen Aarakocra Sneak enters the battle...,[Flying],1,4,False,common,False,2022-06-10,True,0.08
2,Abaddon the Despoiler,5.0,"[2, U, B, R]","['B', 'R', 'U']",True,[Creature],"[Astartes, Warrior]",Trample\nMark of Chaos Ascendant — During your...,"[Mark of Chaos Ascendant, Trample]",5,5,False,mythic,False,2022-10-07,False,2.88
3,Abandoned Outpost,0.0,[0],['No Colors'],False,[Land],[none],Abandoned Outpost enters the battlefield tappe...,[no keywords],non-creature,non-creature,False,common,False,2001-10-01,True,0.14
4,Abandoned Sarcophagus,3.0,[3],['C'],False,[Artifact],[none],You may cast spells that have a cycling abilit...,[no keywords],non-creature,non-creature,False,rare,False,2020-04-17,False,0.09


In [11]:
rarity_ohe = OneHotEncoder()
rarity_encoded = rarity_ohe.fit_transform(df[['rarity']]).toarray()
rarity_ohe.categories_


[array(['common', 'mythic', 'rare', 'special', 'uncommon'], dtype=object)]

In [12]:
column_names = rarity_ohe.get_feature_names_out(['rarity'])
df_encoded = pd.DataFrame(rarity_encoded, columns=column_names)
df_encoded.head()


,rarity_common,rarity_mythic,rarity_rare,rarity_special,rarity_uncommon
0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0


In [13]:
df = pd.concat([df, df_encoded], axis=1)
df.drop(columns='rarity', inplace=True)
df.head()


,name,cmc,mana_cost,colors,legendary,types,sub_types,oracle_text,keywords,power,toughness,textless,reserved,released_at,booster,usd,rarity_common,rarity_mythic,rarity_rare,rarity_special,rarity_uncommon
0,+2 Mace,2.0,"[1, W]",['W'],False,[Artifact],[Equipment],Equipped creature gets +2/+2.\nEquip {3} ({3}:...,[Equip],non-creature,non-creature,False,False,2021-07-23,True,0.02,1.0,0.0,0.0,0.0,0.0
1,Aarakocra Sneak,4.0,"[3, U]",['U'],False,[Creature],"[Bird, Rogue]",Flying\nWhen Aarakocra Sneak enters the battle...,[Flying],1,4,False,False,2022-06-10,True,0.08,1.0,0.0,0.0,0.0,0.0
2,Abaddon the Despoiler,5.0,"[2, U, B, R]","['B', 'R', 'U']",True,[Creature],"[Astartes, Warrior]",Trample\nMark of Chaos Ascendant — During your...,"[Mark of Chaos Ascendant, Trample]",5,5,False,False,2022-10-07,False,2.88,0.0,1.0,0.0,0.0,0.0
3,Abandoned Outpost,0.0,[0],['No Colors'],False,[Land],[none],Abandoned Outpost enters the battlefield tappe...,[no keywords],non-creature,non-creature,False,False,2001-10-01,True,0.14,1.0,0.0,0.0,0.0,0.0
4,Abandoned Sarcophagus,3.0,[3],['C'],False,[Artifact],[none],You may cast spells that have a cycling abilit...,[no keywords],non-creature,non-creature,False,False,2020-04-17,False,0.09,0.0,0.0,1.0,0.0,0.0


In [14]:
df.colors.value_counts()


colors
['W']                   3921
['B']                   3884
['R']                   3848
['U']                   3812
['G']                   3802
                        ... 
['G', 'R', 'U', 'W']       5
['B', 'G', 'U', 'W']       3
['B', 'R', 'U', 'W']       2
['B', 'G', 'R', 'W']       2
['B', 'G', 'R', 'U']       2
Name: count, Length: 34, dtype: int64

In [15]:
df[df['colors']=='[nan]'].head()


,name,cmc,mana_cost,colors,legendary,types,sub_types,oracle_text,keywords,power,toughness,textless,reserved,released_at,booster,usd,rarity_common,rarity_mythic,rarity_rare,rarity_special,rarity_uncommon
17,Aberrant Researcher // Perfected Form,4.0,"[n, a, n]",[nan],False,[Creature],"[Human, Insect, //, Creature, —, Insect, Horror]",NaN,"[Flying, Transform, Mill]",non-creature,non-creature,False,False,2016-04-08,True,0.03,0.0,0.0,0.0,0.0,1.0
97,Accursed Witch // Infectious Curse,4.0,"[n, a, n]",[nan],False,[Creature],"[Human, Shaman, //, Enchantment, —, Aura, Curse]",NaN,"[Transform, Enchant]",non-creature,non-creature,False,False,2016-04-08,True,0.28,0.0,0.0,0.0,0.0,1.0
108,"Aclazotz, Deepest Betrayal // Temple of the Dead",5.0,"[n, a, n]",[nan],True,[Creature],"[Bat, God, //, Land]",NaN,"[Flying, Lifelink, Transform]",non-creature,non-creature,False,False,2023-11-17,True,3.38,0.0,1.0,0.0,0.0,0.0
212,Aetherblade Agent // Gitaxian Mindstinger,2.0,"[n, a, n]",[nan],False,[Creature],"[Human, Rogue, //, Creature, —, Phyrexian, Rogue]",NaN,"[Transform, Deathtouch]",non-creature,non-creature,False,False,2023-04-21,True,0.02,1.0,0.0,0.0,0.0,0.0
269,Afflicted Deserter // Werewolf Ransacker,4.0,"[n, a, n]",[nan],False,[Creature],"[Human, Werewolf, //, Creature, —, Werewolf]",NaN,[Transform],non-creature,non-creature,False,False,2012-02-03,True,0.27,0.0,0.0,0.0,0.0,1.0


In [16]:
df.drop(df[df['colors']=='[nan]'].index, inplace=True)
df.reset_index(drop=True)
df.head()


,name,cmc,mana_cost,colors,legendary,types,sub_types,oracle_text,keywords,power,toughness,textless,reserved,released_at,booster,usd,rarity_common,rarity_mythic,rarity_rare,rarity_special,rarity_uncommon
0,+2 Mace,2.0,"[1, W]",['W'],False,[Artifact],[Equipment],Equipped creature gets +2/+2.\nEquip {3} ({3}:...,[Equip],non-creature,non-creature,False,False,2021-07-23,True,0.02,1.0,0.0,0.0,0.0,0.0
1,Aarakocra Sneak,4.0,"[3, U]",['U'],False,[Creature],"[Bird, Rogue]",Flying\nWhen Aarakocra Sneak enters the battle...,[Flying],1,4,False,False,2022-06-10,True,0.08,1.0,0.0,0.0,0.0,0.0
2,Abaddon the Despoiler,5.0,"[2, U, B, R]","['B', 'R', 'U']",True,[Creature],"[Astartes, Warrior]",Trample\nMark of Chaos Ascendant — During your...,"[Mark of Chaos Ascendant, Trample]",5,5,False,False,2022-10-07,False,2.88,0.0,1.0,0.0,0.0,0.0
3,Abandoned Outpost,0.0,[0],['No Colors'],False,[Land],[none],Abandoned Outpost enters the battlefield tappe...,[no keywords],non-creature,non-creature,False,False,2001-10-01,True,0.14,1.0,0.0,0.0,0.0,0.0
4,Abandoned Sarcophagus,3.0,[3],['C'],False,[Artifact],[none],You may cast spells that have a cycling abilit...,[no keywords],non-creature,non-creature,False,False,2020-04-17,False,0.09,0.0,0.0,1.0,0.0,0.0


In [17]:
df.colors.value_counts()


colors
['W']                   3921
['B']                   3884
['R']                   3848
['U']                   3812
['G']                   3802
                        ... 
['G', 'R', 'U', 'W']       5
['B', 'G', 'U', 'W']       3
['B', 'R', 'U', 'W']       2
['B', 'G', 'R', 'W']       2
['B', 'G', 'R', 'U']       2
Name: count, Length: 33, dtype: int64

In [18]:
df['colors'].replace("['No Colors']", "['N']", inplace=True)
df['colors'].value_counts().head(7)


colors
['W']    3921
['B']    3884
['R']    3848
['U']    3812
['G']    3802
['C']    2371
['N']     906
Name: count, dtype: int64

In [19]:
df['colors']=df['colors'].apply(literal_eval)
df['colors'].head()


0          [W]
1          [U]
2    [B, R, U]
3          [N]
4          [C]
Name: colors, dtype: object

In [20]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 25700 entries, 0 to 26051
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             25700 non-null  object 
 1   cmc              25700 non-null  float64
 2   mana_cost        25700 non-null  object 
 3   colors           25700 non-null  object 
 4   legendary        25700 non-null  bool   
 5   types            25700 non-null  object 
 6   sub_types        25700 non-null  object 
 7   oracle_text      25099 non-null  object 
 8   keywords         25700 non-null  object 
 9   power            25700 non-null  object 
 10  toughness        25700 non-null  object 
 11  textless         25700 non-null  bool   
 12  reserved         25700 non-null  bool   
 13  released_at      25700 non-null  object 
 14  booster          25700 non-null  bool   
 15  usd              25646 non-null  float64
 16  rarity_common    25700 non-null  float64
 17  rarity_mythic    

In [21]:
df.reset_index(drop=True, inplace=True)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25700 entries, 0 to 25699
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             25700 non-null  object 
 1   cmc              25700 non-null  float64
 2   mana_cost        25700 non-null  object 
 3   colors           25700 non-null  object 
 4   legendary        25700 non-null  bool   
 5   types            25700 non-null  object 
 6   sub_types        25700 non-null  object 
 7   oracle_text      25099 non-null  object 
 8   keywords         25700 non-null  object 
 9   power            25700 non-null  object 
 10  toughness        25700 non-null  object 
 11  textless         25700 non-null  bool   
 12  reserved         25700 non-null  bool   
 13  released_at      25700 non-null  object 
 14  booster          25700 non-null  bool   
 15  usd              25646 non-null  float64
 16  rarity_common    25700 non-null  float64
 17  rarity_mythi

In [22]:
colors_mlb = MultiLabelBinarizer()
colors_mlb.fit(df['colors'])
colors_mlb.classes_


array(['B', 'C', 'G', 'N', 'R', 'U', 'W'], dtype=object)

In [23]:
binary_colors = colors_mlb.transform(df['colors'])
colors_df = pd.DataFrame(binary_colors, columns=colors_mlb.classes_)
colors_df.head()


,B,C,G,N,R,U,W
0,0,0,0,0,0,0,1
1,0,0,0,0,0,1,0
2,1,0,0,0,1,1,0
3,0,0,0,1,0,0,0
4,0,1,0,0,0,0,0


In [24]:
df.drop('colors', axis=1, inplace=True)
df.head()


,name,cmc,mana_cost,legendary,types,sub_types,oracle_text,keywords,power,toughness,textless,reserved,released_at,booster,usd,rarity_common,rarity_mythic,rarity_rare,rarity_special,rarity_uncommon
0,+2 Mace,2.0,"[1, W]",False,[Artifact],[Equipment],Equipped creature gets +2/+2.\nEquip {3} ({3}:...,[Equip],non-creature,non-creature,False,False,2021-07-23,True,0.02,1.0,0.0,0.0,0.0,0.0
1,Aarakocra Sneak,4.0,"[3, U]",False,[Creature],"[Bird, Rogue]",Flying\nWhen Aarakocra Sneak enters the battle...,[Flying],1,4,False,False,2022-06-10,True,0.08,1.0,0.0,0.0,0.0,0.0
2,Abaddon the Despoiler,5.0,"[2, U, B, R]",True,[Creature],"[Astartes, Warrior]",Trample\nMark of Chaos Ascendant — During your...,"[Mark of Chaos Ascendant, Trample]",5,5,False,False,2022-10-07,False,2.88,0.0,1.0,0.0,0.0,0.0
3,Abandoned Outpost,0.0,[0],False,[Land],[none],Abandoned Outpost enters the battlefield tappe...,[no keywords],non-creature,non-creature,False,False,2001-10-01,True,0.14,1.0,0.0,0.0,0.0,0.0
4,Abandoned Sarcophagus,3.0,[3],False,[Artifact],[none],You may cast spells that have a cycling abilit...,[no keywords],non-creature,non-creature,False,False,2020-04-17,False,0.09,0.0,0.0,1.0,0.0,0.0


In [25]:
df = pd.concat([df, colors_df], axis=1)
df.head()


,name,cmc,mana_cost,legendary,types,sub_types,oracle_text,keywords,power,toughness,textless,reserved,released_at,booster,usd,rarity_common,rarity_mythic,rarity_rare,rarity_special,rarity_uncommon,B,C,G,N,R,U,W
0,+2 Mace,2.0,"[1, W]",False,[Artifact],[Equipment],Equipped creature gets +2/+2.\nEquip {3} ({3}:...,[Equip],non-creature,non-creature,False,False,2021-07-23,True,0.02,1.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,1
1,Aarakocra Sneak,4.0,"[3, U]",False,[Creature],"[Bird, Rogue]",Flying\nWhen Aarakocra Sneak enters the battle...,[Flying],1,4,False,False,2022-06-10,True,0.08,1.0,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0
2,Abaddon the Despoiler,5.0,"[2, U, B, R]",True,[Creature],"[Astartes, Warrior]",Trample\nMark of Chaos Ascendant — During your...,"[Mark of Chaos Ascendant, Trample]",5,5,False,False,2022-10-07,False,2.88,0.0,1.0,0.0,0.0,0.0,1,0,0,0,1,1,0
3,Abandoned Outpost,0.0,[0],False,[Land],[none],Abandoned Outpost enters the battlefield tappe...,[no keywords],non-creature,non-creature,False,False,2001-10-01,True,0.14,1.0,0.0,0.0,0.0,0.0,0,0,0,1,0,0,0
4,Abandoned Sarcophagus,3.0,[3],False,[Artifact],[none],You may cast spells that have a cycling abilit...,[no keywords],non-creature,non-creature,False,False,2020-04-17,False,0.09,0.0,0.0,1.0,0.0,0.0,0,1,0,0,0,0,0


In [26]:
legend_binarizer = LabelBinarizer()
df['legendary'] = legend_binarizer.fit_transform(df['legendary'])
df['legendary'].head()


0    0
1    0
2    1
3    0
4    0
Name: legendary, dtype: int64

In [27]:
textless_binarizer = LabelBinarizer()
df['textless'] = textless_binarizer.fit_transform(df['textless'])
df['textless'].head()


0    0
1    0
2    0
3    0
4    0
Name: textless, dtype: int64

In [28]:
reserved_binarizer = LabelBinarizer()
df['reserved'] = textless_binarizer.fit_transform(df['reserved'])
df['reserved'].head()


0    0
1    0
2    0
3    0
4    0
Name: reserved, dtype: int64

In [29]:
booster_binarizer = LabelBinarizer()
df['booster'] = textless_binarizer.fit_transform(df['booster'])
df['booster'].head()


0    1
1    1
2    0
3    1
4    0
Name: booster, dtype: int64

In [30]:
df.head()


,name,cmc,mana_cost,legendary,types,sub_types,oracle_text,keywords,power,toughness,textless,reserved,released_at,booster,usd,rarity_common,rarity_mythic,rarity_rare,rarity_special,rarity_uncommon,B,C,G,N,R,U,W
0,+2 Mace,2.0,"[1, W]",0,[Artifact],[Equipment],Equipped creature gets +2/+2.\nEquip {3} ({3}:...,[Equip],non-creature,non-creature,0,0,2021-07-23,1,0.02,1.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,1
1,Aarakocra Sneak,4.0,"[3, U]",0,[Creature],"[Bird, Rogue]",Flying\nWhen Aarakocra Sneak enters the battle...,[Flying],1,4,0,0,2022-06-10,1,0.08,1.0,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0
2,Abaddon the Despoiler,5.0,"[2, U, B, R]",1,[Creature],"[Astartes, Warrior]",Trample\nMark of Chaos Ascendant — During your...,"[Mark of Chaos Ascendant, Trample]",5,5,0,0,2022-10-07,0,2.88,0.0,1.0,0.0,0.0,0.0,1,0,0,0,1,1,0
3,Abandoned Outpost,0.0,[0],0,[Land],[none],Abandoned Outpost enters the battlefield tappe...,[no keywords],non-creature,non-creature,0,0,2001-10-01,1,0.14,1.0,0.0,0.0,0.0,0.0,0,0,0,1,0,0,0
4,Abandoned Sarcophagus,3.0,[3],0,[Artifact],[none],You may cast spells that have a cycling abilit...,[no keywords],non-creature,non-creature,0,0,2020-04-17,0,0.09,0.0,0.0,1.0,0.0,0.0,0,1,0,0,0,0,0


In [31]:
list(df.columns.values)


['name',
 'cmc',
 'mana_cost',
 'legendary',
 'types',
 'sub_types',
 'oracle_text',
 'keywords',
 'power',
 'toughness',
 'textless',
 'reserved',
 'released_at',
 'booster',
 'usd',
 'rarity_common',
 'rarity_mythic',
 'rarity_rare',
 'rarity_special',
 'rarity_uncommon',
 'B',
 'C',
 'G',
 'N',
 'R',
 'U',
 'W']

In [32]:
df = df[['name',
 'cmc',
 'mana_cost',
 'B',
 'C',
 'G',
 'N',
 'R',
 'U',
 'W',
 'legendary',
 'types',
 'sub_types',
 'oracle_text',
 'keywords',
 'power',
 'toughness',
 'textless',
 'reserved',
 'rarity_common',
 'rarity_mythic',
 'rarity_rare',
 'rarity_special',
 'rarity_uncommon',
 'released_at',
 'booster',
 'usd']]
df.head()


,name,cmc,mana_cost,B,C,G,N,R,U,W,legendary,types,sub_types,oracle_text,keywords,power,toughness,textless,reserved,rarity_common,rarity_mythic,rarity_rare,rarity_special,rarity_uncommon,released_at,booster,usd
0,+2 Mace,2.0,"[1, W]",0,0,0,0,0,0,1,0,[Artifact],[Equipment],Equipped creature gets +2/+2.\nEquip {3} ({3}:...,[Equip],non-creature,non-creature,0,0,1.0,0.0,0.0,0.0,0.0,2021-07-23,1,0.02
1,Aarakocra Sneak,4.0,"[3, U]",0,0,0,0,0,1,0,0,[Creature],"[Bird, Rogue]",Flying\nWhen Aarakocra Sneak enters the battle...,[Flying],1,4,0,0,1.0,0.0,0.0,0.0,0.0,2022-06-10,1,0.08
2,Abaddon the Despoiler,5.0,"[2, U, B, R]",1,0,0,0,1,1,0,1,[Creature],"[Astartes, Warrior]",Trample\nMark of Chaos Ascendant — During your...,"[Mark of Chaos Ascendant, Trample]",5,5,0,0,0.0,1.0,0.0,0.0,0.0,2022-10-07,0,2.88
3,Abandoned Outpost,0.0,[0],0,0,0,1,0,0,0,0,[Land],[none],Abandoned Outpost enters the battlefield tappe...,[no keywords],non-creature,non-creature,0,0,1.0,0.0,0.0,0.0,0.0,2001-10-01,1,0.14
4,Abandoned Sarcophagus,3.0,[3],0,1,0,0,0,0,0,0,[Artifact],[none],You may cast spells that have a cycling abilit...,[no keywords],non-creature,non-creature,0,0,0.0,0.0,1.0,0.0,0.0,2020-04-17,0,0.09


In [33]:
df[['power', 'toughness']].head()


,power,toughness
0,non-creature,non-creature
1,1,4
2,5,5
3,non-creature,non-creature
4,non-creature,non-creature


#### I deicded that non-creatue cards will now have power toughness of 0


In [34]:
df['power'] = df['power'].replace('non-creature', 0)
df['toughness'] = df['toughness'].replace('non-creature', 0)
df[['power', 'toughness']].head()


,power,toughness
0,0,0
1,1,4
2,5,5
3,0,0
4,0,0


#### i decided that mana cost maybe useless at this point since we have cmc and the colors


In [35]:
df.drop(columns='mana_cost', inplace=True)
df.head()


,name,cmc,B,C,G,N,R,U,W,legendary,types,sub_types,oracle_text,keywords,power,toughness,textless,reserved,rarity_common,rarity_mythic,rarity_rare,rarity_special,rarity_uncommon,released_at,booster,usd
0,+2 Mace,2.0,0,0,0,0,0,0,1,0,[Artifact],[Equipment],Equipped creature gets +2/+2.\nEquip {3} ({3}:...,[Equip],0,0,0,0,1.0,0.0,0.0,0.0,0.0,2021-07-23,1,0.02
1,Aarakocra Sneak,4.0,0,0,0,0,0,1,0,0,[Creature],"[Bird, Rogue]",Flying\nWhen Aarakocra Sneak enters the battle...,[Flying],1,4,0,0,1.0,0.0,0.0,0.0,0.0,2022-06-10,1,0.08
2,Abaddon the Despoiler,5.0,1,0,0,0,1,1,0,1,[Creature],"[Astartes, Warrior]",Trample\nMark of Chaos Ascendant — During your...,"[Mark of Chaos Ascendant, Trample]",5,5,0,0,0.0,1.0,0.0,0.0,0.0,2022-10-07,0,2.88
3,Abandoned Outpost,0.0,0,0,0,1,0,0,0,0,[Land],[none],Abandoned Outpost enters the battlefield tappe...,[no keywords],0,0,0,0,1.0,0.0,0.0,0.0,0.0,2001-10-01,1,0.14
4,Abandoned Sarcophagus,3.0,0,1,0,0,0,0,0,0,[Artifact],[none],You may cast spells that have a cycling abilit...,[no keywords],0,0,0,0,0.0,0.0,1.0,0.0,0.0,2020-04-17,0,0.09


In [36]:
df[df['types'].apply(lambda x: '//' in x)]['oracle_text']


269      NaN
454      NaN
501      NaN
797      NaN
1013     NaN
        ... 
24718    NaN
24806    NaN
24829    NaN
24903    NaN
25660    NaN
Name: oracle_text, Length: 155, dtype: object

#### The cards have no oracle text thus they must be dropped, which are transform cards noted by '//'

In [37]:
df.drop(df[df['types'].apply(lambda x: '//' in x)].index, inplace=True)
df.reset_index(drop=True, inplace=True)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25545 entries, 0 to 25544
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             25545 non-null  object 
 1   cmc              25545 non-null  float64
 2   B                25545 non-null  int64  
 3   C                25545 non-null  int64  
 4   G                25545 non-null  int64  
 5   N                25545 non-null  int64  
 6   R                25545 non-null  int64  
 7   U                25545 non-null  int64  
 8   W                25545 non-null  int64  
 9   legendary        25545 non-null  int64  
 10  types            25545 non-null  object 
 11  sub_types        25545 non-null  object 
 12  oracle_text      25099 non-null  object 
 13  keywords         25545 non-null  object 
 14  power            25545 non-null  object 
 15  toughness        25545 non-null  object 
 16  textless         25545 non-null  int64  
 17  reserved    

#### Initial code to preprocess primary types

In [38]:
'''
types_mlb=MultiLabelBinarizer()
types_mlb.fit(df['types'])
types_mlb.classes_
'''


"\ntypes_mlb=MultiLabelBinarizer()\ntypes_mlb.fit(df['types'])\ntypes_mlb.classes_\n"

In [39]:
'''
binary_types = types_mlb.transform(df['types'])
types_df = pd.DataFrame(binary_types, columns=types_mlb.classes_)
types_df.head()
'''


"\nbinary_types = types_mlb.transform(df['types'])\ntypes_df = pd.DataFrame(binary_types, columns=types_mlb.classes_)\ntypes_df.head()\n"

In [40]:
'''
df.drop('types', axis=1, inplace=True)
df.head()
'''


"\ndf.drop('types', axis=1, inplace=True)\ndf.head()\n"

In [41]:
'''
df = pd.concat([df, types_df], axis=1)
df.head()
'''


'\ndf = pd.concat([df, types_df], axis=1)\ndf.head()\n'

In [42]:
df = df[['name',
 'cmc',
 'B',
 'C',
 'G',
 'N',
 'R',
 'U',
 'W',
 'legendary',
 'types',
 'sub_types',
 'oracle_text',
 'keywords',
 'power',
 'toughness',
 'textless',
 'reserved',
 'rarity_common',
 'rarity_mythic',
 'rarity_rare',
 'rarity_special',
 'rarity_uncommon',
 'released_at',
 'booster',
 'usd'
]]
df.head()


,name,cmc,B,C,G,N,R,U,W,legendary,types,sub_types,oracle_text,keywords,power,toughness,textless,reserved,rarity_common,rarity_mythic,rarity_rare,rarity_special,rarity_uncommon,released_at,booster,usd
0,+2 Mace,2.0,0,0,0,0,0,0,1,0,[Artifact],[Equipment],Equipped creature gets +2/+2.\nEquip {3} ({3}:...,[Equip],0,0,0,0,1.0,0.0,0.0,0.0,0.0,2021-07-23,1,0.02
1,Aarakocra Sneak,4.0,0,0,0,0,0,1,0,0,[Creature],"[Bird, Rogue]",Flying\nWhen Aarakocra Sneak enters the battle...,[Flying],1,4,0,0,1.0,0.0,0.0,0.0,0.0,2022-06-10,1,0.08
2,Abaddon the Despoiler,5.0,1,0,0,0,1,1,0,1,[Creature],"[Astartes, Warrior]",Trample\nMark of Chaos Ascendant — During your...,"[Mark of Chaos Ascendant, Trample]",5,5,0,0,0.0,1.0,0.0,0.0,0.0,2022-10-07,0,2.88
3,Abandoned Outpost,0.0,0,0,0,1,0,0,0,0,[Land],[none],Abandoned Outpost enters the battlefield tappe...,[no keywords],0,0,0,0,1.0,0.0,0.0,0.0,0.0,2001-10-01,1,0.14
4,Abandoned Sarcophagus,3.0,0,1,0,0,0,0,0,0,[Artifact],[none],You may cast spells that have a cycling abilit...,[no keywords],0,0,0,0,0.0,0.0,1.0,0.0,0.0,2020-04-17,0,0.09


In [43]:
df[['sub_types', 'keywords']].head()


,sub_types,keywords
0,[Equipment],[Equip]
1,"[Bird, Rogue]",[Flying]
2,"[Astartes, Warrior]","[Mark of Chaos Ascendant, Trample]"
3,[none],[no keywords]
4,[none],[no keywords]


####


In [44]:
subtypes_column = df['sub_types']

subtype_model = Word2Vec(sentences=subtypes_column, vector_size=100, window=5, min_count=1, workers=4)

subtype_model.save('sub_type.model')

In [45]:
subtype_model.wv.index_to_key

['none',
 'Human',
 'Aura',
 'Warrior',
 'Wizard',
 'Soldier',
 'Spirit',
 'Elf',
 'Cleric',
 'Zombie',
 'Elemental',
 'Beast',
 'Shaman',
 'Equipment',
 'Phyrexian',
 'Knight',
 'Goblin',
 'Rogue',
 'Vampire',
 'Bird',
 'Dragon',
 'Druid',
 'Merfolk',
 'Horror',
 'Cat',
 'Angel',
 'Giant',
 'Artificer',
 'Insect',
 'Scout',
 'Construct',
 'Dinosaur',
 'Demon',
 'Wall',
 'Faerie',
 'Pirate',
 'Golem',
 'Shapeshifter',
 'Eldrazi',
 'Ogre',
 'Dwarf',
 '//',
 '—',
 'Berserker',
 'Sliver',
 'Noble',
 'Dog',
 'Avatar',
 'Snake',
 'Monk',
 'Wurm',
 'Adventure',
 'Saga',
 'Treefolk',
 'Drake',
 'Advisor',
 'Minotaur',
 'Archer',
 'Ally',
 'Arcane',
 'Illusion',
 'Orc',
 'Vehicle',
 'Warlock',
 'Kor',
 'Wolf',
 'Spider',
 'Rat',
 'Assassin',
 'Sphinx',
 'Nightmare',
 'Vedalken',
 'Centaur',
 'Elephant',
 'Lizard',
 'Skeleton',
 'Samurai',
 'Drone',
 'Kithkin',
 'Plant',
 'Fungus',
 'Hydra',
 'Rebel',
 'Mutant',
 'Spellshaper',
 'Viashino',
 'Djinn',
 'Griffin',
 'Sorcery',
 'Troll',
 'Instant'

In [46]:
astartes_warrior_embedding = subtype_model.wv['Astartes'] + subtype_model.wv['Warrior']
astartes_warrior_embedding

array([-9.36579630e-02,  2.17962056e-01, -3.22350599e-02, -3.15825164e-01,
        3.53924990e-01, -3.38442028e-01,  4.73341681e-02,  4.37777907e-01,
        1.98296458e-02, -4.07165259e-01,  7.51117989e-02, -2.70686865e-01,
       -9.45348293e-02,  1.47091568e-01,  2.97850110e-02, -8.04776251e-02,
        1.55506865e-03,  1.98411644e-02, -1.15903594e-01, -3.66577983e-01,
        3.16948108e-02, -5.56727983e-02,  1.06521383e-01, -2.90573537e-01,
       -1.64630830e-01, -5.69435768e-02,  7.23281992e-04, -1.71504587e-01,
        5.97726144e-02,  1.14024311e-01,  3.20355088e-01, -2.07963690e-01,
        4.02578413e-02, -3.88161957e-01,  2.54065404e-03,  4.23538566e-01,
        3.28523889e-02,  1.33192614e-01,  3.86524051e-02, -2.39107624e-01,
        2.71387339e-01, -2.06484139e-01,  2.90665347e-02, -5.93470745e-02,
        1.40763689e-02, -1.06658563e-01, -1.11758292e-01, -2.40113419e-02,
        1.35082766e-01,  1.39162540e-01,  4.09533709e-01, -2.09035203e-01,
        2.29420602e-01,  

In [47]:
df['sub_type_embedding'] = df['sub_types'].apply(lambda types: np.sum([subtype_model.wv[word] for word in types], axis=0))
df['sub_type_embedding'].head()

0    [-0.029019738, 0.056784265, -0.0180905, -0.080...
1    [-0.09787789, 0.25772893, -0.042671315, -0.395...
2    [-0.09365796, 0.21796206, -0.03223506, -0.3158...
3    [-0.00053622725, 0.00023643136, 0.0051033497, ...
4    [-0.00053622725, 0.00023643136, 0.0051033497, ...
Name: sub_type_embedding, dtype: object

In [48]:
df.head()

,name,cmc,B,C,G,N,R,U,W,legendary,types,sub_types,oracle_text,keywords,power,toughness,textless,reserved,rarity_common,rarity_mythic,rarity_rare,rarity_special,rarity_uncommon,released_at,booster,usd,sub_type_embedding
0,+2 Mace,2.0,0,0,0,0,0,0,1,0,[Artifact],[Equipment],Equipped creature gets +2/+2.\nEquip {3} ({3}:...,[Equip],0,0,0,0,1.0,0.0,0.0,0.0,0.0,2021-07-23,1,0.02,"[-0.029019738, 0.056784265, -0.0180905, -0.080..."
1,Aarakocra Sneak,4.0,0,0,0,0,0,1,0,0,[Creature],"[Bird, Rogue]",Flying\nWhen Aarakocra Sneak enters the battle...,[Flying],1,4,0,0,1.0,0.0,0.0,0.0,0.0,2022-06-10,1,0.08,"[-0.09787789, 0.25772893, -0.042671315, -0.395..."
2,Abaddon the Despoiler,5.0,1,0,0,0,1,1,0,1,[Creature],"[Astartes, Warrior]",Trample\nMark of Chaos Ascendant — During your...,"[Mark of Chaos Ascendant, Trample]",5,5,0,0,0.0,1.0,0.0,0.0,0.0,2022-10-07,0,2.88,"[-0.09365796, 0.21796206, -0.03223506, -0.3158..."
3,Abandoned Outpost,0.0,0,0,0,1,0,0,0,0,[Land],[none],Abandoned Outpost enters the battlefield tappe...,[no keywords],0,0,0,0,1.0,0.0,0.0,0.0,0.0,2001-10-01,1,0.14,"[-0.00053622725, 0.00023643136, 0.0051033497, ..."
4,Abandoned Sarcophagus,3.0,0,1,0,0,0,0,0,0,[Artifact],[none],You may cast spells that have a cycling abilit...,[no keywords],0,0,0,0,0.0,0.0,1.0,0.0,0.0,2020-04-17,0,0.09,"[-0.00053622725, 0.00023643136, 0.0051033497, ..."


In [49]:
keywords_column = df['keywords']

key_model = Word2Vec(sentences=keywords_column, vector_size=100, window=5, min_count=1, workers=4)

key_model.save('keywords.model')

In [50]:
key_model.wv.index_to_key

['no keywords',
 'Flying',
 'Enchant',
 'Trample',
 'Haste',
 'Vigilance',
 'Equip',
 'Flash',
 'Mill',
 'Scry',
 'First strike',
 'Cycling',
 'Defender',
 'Reach',
 'Treasure',
 'Lifelink',
 'Kicker',
 'Menace',
 'Deathtouch',
 'Protection',
 'Flashback',
 'Morph',
 'Landwalk',
 'Landfall',
 'Fight',
 'Partner',
 'Devoid',
 'Food',
 'Threshold',
 'Ward',
 'Crew',
 'Convoke',
 'Indestructible',
 'Cumulative upkeep',
 'Double strike',
 'Proliferate',
 'Hexproof',
 'Suspend',
 'Surveil',
 'Goad',
 'Madness',
 'Investigate',
 'Amass',
 'Domain',
 'Typecycling',
 'Unearth',
 'Landcycling',
 'Echo',
 'Explore',
 'Foretell',
 'Partner with',
 'Delirium',
 'Changeling',
 'Prowess',
 'Infect',
 'Heroic',
 'Swampwalk',
 'Transform',
 'Fear',
 'Buyback',
 'Venture into the dungeon',
 'Role token',
 'Affinity',
 'Channel',
 'Bushido',
 'Shadow',
 'Toxic',
 'Bestow',
 'Shroud',
 'Cascade',
 'Exalted',
 'Metalcraft',
 'Constellation',
 'Mutate',
 'Islandwalk',
 'Incubate',
 'Ninjutsu',
 'Monstrosit

In [51]:
Mark_of_Chaos_Ascendant_Trample_emb = key_model.wv['Mark of Chaos Ascendant'] + key_model.wv['Trample']
Mark_of_Chaos_Ascendant_Trample_emb

array([-0.17963548,  0.26810688, -0.03803291, -0.02853281,  0.10376821,
       -0.2745203 ,  0.11371736,  0.4349641 , -0.2648925 , -0.31579363,
       -0.02104216, -0.2982124 ,  0.09528075, -0.0311017 ,  0.21923794,
        0.00516862,  0.19984908, -0.05230082, -0.01276262, -0.30305028,
        0.05683678, -0.13315946,  0.01609403, -0.12802136, -0.10824469,
        0.15675206, -0.34806976,  0.03609639,  0.00719485,  0.03696788,
        0.1712847 , -0.01666711, -0.13604413, -0.26551223,  0.10900415,
        0.28732502,  0.14705254, -0.0401895 , -0.05754524, -0.08344378,
        0.21278152, -0.33661935, -0.1179877 ,  0.1062567 ,  0.23338193,
       -0.17111836, -0.04540643, -0.17968364,  0.12923813, -0.03693963,
        0.12300527, -0.21087393, -0.07956535, -0.00575185,  0.05161396,
        0.19614017,  0.04613553,  0.12894922,  0.00766611, -0.00450509,
       -0.08628409, -0.02247614,  0.11983539,  0.00689336, -0.016035  ,
        0.19115853,  0.0294153 ,  0.18800054, -0.13115603,  0.05

In [52]:
df['keywords_embedding'] = df['keywords'].apply(lambda keys: np.sum([key_model.wv[word] for word in keys], axis=0))
df['keywords_embedding'].head()

0    [-0.09154848, 0.1352897, -0.02126616, -0.01374...
1    [-0.2023593, 0.27415875, -0.03263983, -0.02227...
2    [-0.17963548, 0.26810688, -0.03803291, -0.0285...
3    [-0.00053622725, 0.00023643136, 0.0051033497, ...
4    [-0.00053622725, 0.00023643136, 0.0051033497, ...
Name: keywords_embedding, dtype: object

In [53]:
df.columns

Index(['name', 'cmc', 'B', 'C', 'G', 'N', 'R', 'U', 'W', 'legendary', 'types',
       'sub_types', 'oracle_text', 'keywords', 'power', 'toughness',
       'textless', 'reserved', 'rarity_common', 'rarity_mythic', 'rarity_rare',
       'rarity_special', 'rarity_uncommon', 'released_at', 'booster', 'usd',
       'sub_type_embedding', 'keywords_embedding'],
      dtype='object')

In [54]:
df.columns

Index(['name', 'cmc', 'B', 'C', 'G', 'N', 'R', 'U', 'W', 'legendary', 'types',
       'sub_types', 'oracle_text', 'keywords', 'power', 'toughness',
       'textless', 'reserved', 'rarity_common', 'rarity_mythic', 'rarity_rare',
       'rarity_special', 'rarity_uncommon', 'released_at', 'booster', 'usd',
       'sub_type_embedding', 'keywords_embedding'],
      dtype='object')

In [55]:
df = df[['name', 'cmc', 'B', 'C', 'G', 'N', 'R', 'U', 'W', 'legendary',
         'types', 'sub_type_embedding', 'keywords_embedding',
       'sub_types', 'keywords', 'power', 'toughness', 'textless', 'reserved',
       'rarity_common', 'rarity_mythic', 'rarity_rare', 'rarity_special',
       'rarity_uncommon', 'released_at', 'booster', 'oracle_text', 'usd',
      ]]

In [56]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df.drop(columns=['keywords', 'sub_types'], inplace=True)
df.head(1)

,name,cmc,B,C,G,N,R,U,W,legendary,types,sub_type_embedding,keywords_embedding,power,toughness,textless,reserved,rarity_common,rarity_mythic,rarity_rare,rarity_special,rarity_uncommon,released_at,booster,oracle_text,usd
0,+2 Mace,2.0,0,0,0,0,0,0,1,0,[Artifact],"[-0.029019738, 0.056784265, -0.0180905, -0.080...","[-0.09154848, 0.1352897, -0.02126616, -0.01374...",0,0,0,0,1.0,0.0,0.0,0.0,0.0,2021-07-23,1,Equipped creature gets +2/+2.\nEquip {3} ({3}:...,0.02


#### I feel that the types should also be embedded at this point

In [57]:
types_column = df['types']

types_model = Word2Vec(sentences=types_column, vector_size=100, window=5, min_count=1, workers=4)

types_model.save('types.model')

In [58]:
types_model.wv.index_to_key

['Creature',
 'Instant',
 'Enchantment',
 'Sorcery',
 'Artifact',
 'Land',
 'Planeswalker',
 'Snow',
 'Tribal',
 'World',
 'Basic']

In [59]:
df['types_embedding'] = df['types'].apply(lambda types: np.sum([types_model.wv[word] for word in types], axis=0))
df['types_embedding'].head()

0    [-0.0073175016, 0.0012215265, -0.0072613307, -...
1    [-0.0004160081, 0.00030767015, 0.007978165, 0....
2    [-0.0004160081, 0.00030767015, 0.007978165, 0....
3    [-0.008775484, 0.0022368848, 0.0009483664, -0....
4    [-0.0073175016, 0.0012215265, -0.0072613307, -...
Name: types_embedding, dtype: object

In [60]:
df.head()

,name,cmc,B,C,G,N,R,U,W,legendary,types,sub_type_embedding,keywords_embedding,power,toughness,textless,reserved,rarity_common,rarity_mythic,rarity_rare,rarity_special,rarity_uncommon,released_at,booster,oracle_text,usd,types_embedding
0,+2 Mace,2.0,0,0,0,0,0,0,1,0,[Artifact],"[-0.029019738, 0.056784265, -0.0180905, -0.080...","[-0.09154848, 0.1352897, -0.02126616, -0.01374...",0,0,0,0,1.0,0.0,0.0,0.0,0.0,2021-07-23,1,Equipped creature gets +2/+2.\nEquip {3} ({3}:...,0.02,"[-0.0073175016, 0.0012215265, -0.0072613307, -..."
1,Aarakocra Sneak,4.0,0,0,0,0,0,1,0,0,[Creature],"[-0.09787789, 0.25772893, -0.042671315, -0.395...","[-0.2023593, 0.27415875, -0.03263983, -0.02227...",1,4,0,0,1.0,0.0,0.0,0.0,0.0,2022-06-10,1,Flying\nWhen Aarakocra Sneak enters the battle...,0.08,"[-0.0004160081, 0.00030767015, 0.007978165, 0...."
2,Abaddon the Despoiler,5.0,1,0,0,0,1,1,0,1,[Creature],"[-0.09365796, 0.21796206, -0.03223506, -0.3158...","[-0.17963548, 0.26810688, -0.03803291, -0.0285...",5,5,0,0,0.0,1.0,0.0,0.0,0.0,2022-10-07,0,Trample\nMark of Chaos Ascendant — During your...,2.88,"[-0.0004160081, 0.00030767015, 0.007978165, 0...."
3,Abandoned Outpost,0.0,0,0,0,1,0,0,0,0,[Land],"[-0.00053622725, 0.00023643136, 0.0051033497, ...","[-0.00053622725, 0.00023643136, 0.0051033497, ...",0,0,0,0,1.0,0.0,0.0,0.0,0.0,2001-10-01,1,Abandoned Outpost enters the battlefield tappe...,0.14,"[-0.008775484, 0.0022368848, 0.0009483664, -0...."
4,Abandoned Sarcophagus,3.0,0,1,0,0,0,0,0,0,[Artifact],"[-0.00053622725, 0.00023643136, 0.0051033497, ...","[-0.00053622725, 0.00023643136, 0.0051033497, ...",0,0,0,0,0.0,0.0,1.0,0.0,0.0,2020-04-17,0,You may cast spells that have a cycling abilit...,0.09,"[-0.0073175016, 0.0012215265, -0.0072613307, -..."


In [61]:
df.columns

Index(['name', 'cmc', 'B', 'C', 'G', 'N', 'R', 'U', 'W', 'legendary', 'types',
       'sub_type_embedding', 'keywords_embedding', 'power', 'toughness',
       'textless', 'reserved', 'rarity_common', 'rarity_mythic', 'rarity_rare',
       'rarity_special', 'rarity_uncommon', 'released_at', 'booster',
       'oracle_text', 'usd', 'types_embedding'],
      dtype='object')

In [62]:
df = df[['name', 'cmc', 'B', 'C', 'G', 'N', 'R', 'U', 'W', 'legendary', 'types', 'types_embedding',
       'sub_type_embedding', 'keywords_embedding', 'power', 'toughness',
       'textless', 'reserved', 'rarity_common', 'rarity_mythic', 'rarity_rare',
       'rarity_special', 'rarity_uncommon', 'released_at', 'booster',
       'oracle_text', 'usd']]
df.drop(columns='types', inplace=True)

In [63]:
df.head(1)

,name,cmc,B,C,G,N,R,U,W,legendary,types_embedding,sub_type_embedding,keywords_embedding,power,toughness,textless,reserved,rarity_common,rarity_mythic,rarity_rare,rarity_special,rarity_uncommon,released_at,booster,oracle_text,usd
0,+2 Mace,2.0,0,0,0,0,0,0,1,0,"[-0.0073175016, 0.0012215265, -0.0072613307, -...","[-0.029019738, 0.056784265, -0.0180905, -0.080...","[-0.09154848, 0.1352897, -0.02126616, -0.01374...",0,0,0,0,1.0,0.0,0.0,0.0,0.0,2021-07-23,1,Equipped creature gets +2/+2.\nEquip {3} ({3}:...,0.02


In [64]:
pd.options.display.max_rows = 10
pd.options.display.max_columns = 10
pd.options.display.max_colwidth = 50
df[df['oracle_text'].isna()][['name','textless', 'oracle_text']]

,name,textless,oracle_text
176,Aegis Turtle,0,NaN
274,Ageless Guardian,0,NaN
363,"Akki Lavarunner // Tok-Tok, Volcano Born",0,NaN
413,Alaborn Trooper,0,NaN
486,Alpha Myr,0,NaN
...,...,...,...
25302,"Yargle, Glutton of Urborg",0,NaN
25349,Yoked Ox,0,NaN
25375,Young Blue Dragon // Sand Augury,0,NaN
25378,Young Red Dragon // Bathe in Gold,0,NaN


In [65]:
df[df['name'].str.contains('//')][['name','textless', 'oracle_text']]

,name,textless,oracle_text
363,"Akki Lavarunner // Tok-Tok, Volcano Born",0,NaN
534,Amethyst Dragon // Explosive Crystal,0,NaN
716,Animating Faerie // Bring to Life,0,NaN
805,Aquatic Alchemist // Bubble Up,0,NaN
981,Ardenvale Tactician // Dizzying Swoop,0,NaN
...,...,...,...
23330,Two-Headed Hunter // Twice the Rage,0,NaN
23775,Vantress Transmuter // Croaking Curse,0,NaN
25135,Woodland Acolyte // Mend the Wilds,0,NaN
25375,Young Blue Dragon // Sand Augury,0,NaN


#### Drop all dual sides and adventure cards

In [66]:
null_texts_ind = df[df['name'].str.contains('//')][['name','oracle_text']].index

In [67]:
df.drop(columns='textless', inplace=True)

In [68]:
df.drop(null_texts_ind, inplace=True)
df.reset_index(drop=True)

,name,cmc,B,C,G,...,rarity_uncommon,released_at,booster,oracle_text,usd
0,+2 Mace,2.0,0,0,0,...,0.0,2021-07-23,1,Equipped creature gets +2/+2.\nEquip {3} ({3}:...,0.02
1,Aarakocra Sneak,4.0,0,0,0,...,0.0,2022-06-10,1,Flying\nWhen Aarakocra Sneak enters the battle...,0.08
2,Abaddon the Despoiler,5.0,1,0,0,...,0.0,2022-10-07,0,Trample\nMark of Chaos Ascendant — During your...,2.88
3,Abandoned Outpost,0.0,0,0,0,...,0.0,2001-10-01,1,Abandoned Outpost enters the battlefield tappe...,0.14
4,Abandoned Sarcophagus,3.0,0,1,0,...,0.0,2020-04-17,0,You may cast spells that have a cycling abilit...,0.09
...,...,...,...,...,...,...,...,...,...,...,...
25419,Zurgo Helmsmasher,5.0,1,0,0,...,0.0,2014-09-26,1,Haste\nZurgo Helmsmasher attacks each combat i...,0.42
25420,Zur's Weirding,4.0,0,0,0,...,0.0,2005-07-29,1,Players play with their hands revealed.\nIf a ...,0.82
25421,Zur the Enchanter,4.0,1,0,0,...,0.0,2022-07-08,1,"Flying\nWhenever Zur the Enchanter attacks, yo...",0.53
25422,"Zurzoth, Chaos Rider",3.0,0,0,0,...,0.0,2022-04-29,0,Whenever an opponent draws their first card ea...,0.24


In [69]:
df[df['oracle_text'].isna()].head()

,name,cmc,B,C,G,...,rarity_uncommon,released_at,booster,oracle_text,usd
176,Aegis Turtle,1.0,0,0,0,...,0.0,2020-07-17,1,NaN,0.07
274,Ageless Guardian,2.0,0,0,0,...,0.0,2021-04-23,1,NaN,0.02
413,Alaborn Trooper,3.0,0,0,0,...,0.0,1998-06-24,1,NaN,0.10
486,Alpha Myr,2.0,0,1,0,...,0.0,2003-10-02,1,NaN,0.23
488,Alpha Tyrranax,6.0,0,0,1,...,0.0,2010-10-01,1,NaN,0.04


In [70]:
df['oracle_text'].fillna('notext', inplace=True)
df[df['oracle_text']=='notext']

,name,cmc,B,C,G,...,rarity_uncommon,released_at,booster,oracle_text,usd
176,Aegis Turtle,1.0,0,0,0,...,0.0,2020-07-17,1,notext,0.07
274,Ageless Guardian,2.0,0,0,0,...,0.0,2021-04-23,1,notext,0.02
413,Alaborn Trooper,3.0,0,0,0,...,0.0,1998-06-24,1,notext,0.10
486,Alpha Myr,2.0,0,1,0,...,0.0,2003-10-02,1,notext,0.23
488,Alpha Tyrranax,6.0,0,0,1,...,0.0,2010-10-01,1,notext,0.04
...,...,...,...,...,...,...,...,...,...,...,...
25250,Wu Infantry,2.0,0,0,0,...,0.0,1999-05-01,1,notext,1.92
25301,Yargle and Multani,6.0,1,0,1,...,0.0,2023-04-21,1,notext,0.74
25302,"Yargle, Glutton of Urborg",5.0,1,0,0,...,1.0,2023-08-04,1,notext,0.04
25349,Yoked Ox,1.0,0,0,0,...,0.0,2019-07-12,1,notext,0.06


In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25424 entries, 0 to 25544
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   name                25424 non-null  object 
 1   cmc                 25424 non-null  float64
 2   B                   25424 non-null  int64  
 3   C                   25424 non-null  int64  
 4   G                   25424 non-null  int64  
 5   N                   25424 non-null  int64  
 6   R                   25424 non-null  int64  
 7   U                   25424 non-null  int64  
 8   W                   25424 non-null  int64  
 9   legendary           25424 non-null  int64  
 10  types_embedding     25424 non-null  object 
 11  sub_type_embedding  25424 non-null  object 
 12  keywords_embedding  25424 non-null  object 
 13  power               25424 non-null  object 
 14  toughness           25424 non-null  object 
 15  reserved            25424 non-null  int64  
 16  rarity_co

In [72]:
pd.options.display.max_colwidth = 3000
df[df['name']=='Shigeki, Jukai Visionary']['oracle_text']

19371    {1}{G}, {T}, Return Shigeki, Jukai Visionary to its owner's hand: Reveal the top four cards of your library. You may put a land card from among them onto the battlefield tapped. Put the rest into your graveyard.\nChannel — {X}{X}{G}{G}, Discard Shigeki: Return X target nonlegendary cards from your graveyard to your hand.
Name: oracle_text, dtype: object

In [73]:
df.sample(1)

,name,cmc,B,C,G,...,rarity_uncommon,released_at,booster,oracle_text,usd
1346,Aven Courier,2.0,0,0,0,...,0.0,2022-04-29,0,"Flying\nWhenever Aven Courier attacks, choose a counter on a permanent you control. Put a counter of that kind on target permanent you control if it doesn't have a counter of that kind on it.",0.13


In [74]:
df[df['name']=='Prophet of Distortion']['oracle_text']

16502    Devoid (This card has no color.)\n{3}{C}: Draw a card. ({C} represents colorless mana.)
Name: oracle_text, dtype: object

In [75]:
df[df['name'] == 'Nicol Bolas, Planeswalker']['oracle_text']

14739    +3: Destroy target noncreature permanent.\n−2: Gain control of target creature.\n−9: Nicol Bolas, Planeswalker deals 7 damage to target player or planeswalker. That player or that planeswalker's controller discards seven cards, then sacrifices seven permanents.
Name: oracle_text, dtype: object

In [76]:
def process_oracle_text(oracle_text):

    color_mapping = {'{R}': 'Red ', '{U}': 'Blue ', '{G}': 'Green ', '{B}': 'Black ', '{W}': 'White ',
    '{C}': 'Colorless '}

    for token, color in color_mapping.items():
        oracle_text = oracle_text.replace(token, color)

    oracle_text = re.sub(r'\.', '. ', oracle_text)

    oracle_text = re.sub(r'{(\d+)}', lambda x:f'{x.group(1)} ', oracle_text)

    oracle_text = re.sub(r'{T}', 'Tap', oracle_text)

    oracle_text = re.sub(r'{X}', 'X ', oracle_text)

    oracle_text = re.sub("[\(\[].*?[\)\]]", "", oracle_text)

    return oracle_text

In [77]:
card_text = "{1}{G}, {T}, Return Shigeki, Jukai Visionary to its owner's hand: Reveal the top four cards of your library.You may put a land card from among them onto the battlefield tapped. Put the rest into your graveyard.\nChannel — {X}{X}{G}{G}, Discard Shigeki: Return X target nonlegendary cards from your graveyard to your hand."

processed_card_text = process_oracle_text(card_text)
processed_card_text

"1 Green , Tap, Return Shigeki, Jukai Visionary to its owner's hand: Reveal the top four cards of your library. You may put a land card from among them onto the battlefield tapped.  Put the rest into your graveyard. \nChannel — X X Green Green , Discard Shigeki: Return X target nonlegendary cards from your graveyard to your hand. "

In [78]:
df['oracle_text'] = df['oracle_text'].apply(process_oracle_text)

In [79]:
df[df['name']=='Vampire Nighthawk']['oracle_text']

23728    Flying, deathtouch, lifelink
Name: oracle_text, dtype: object

In [80]:
df[df['name']=='Aarakocra Sneak']['oracle_text']

1    Flying\nWhen Aarakocra Sneak enters the battlefield, you take the initiative. 
Name: oracle_text, dtype: object

In [81]:
df[df['name']=='+2 Mace']['oracle_text']

0    Equipped creature gets +2/+2. \nEquip 3  
Name: oracle_text, dtype: object

In [82]:
df[df['name']=='The Meathook Massacre']

,name,cmc,B,C,G,...,rarity_uncommon,released_at,booster,oracle_text,usd
22310,The Meathook Massacre,2.0,1,0,0,...,0.0,2021-09-24,1,"When The Meathook Massacre enters the battlefield, each creature gets -X/-X until end of turn. \nWhenever a creature you control dies, each opponent loses 1 life. \nWhenever a creature an opponent controls dies, you gain 1 life.",30.68


In [83]:
df.columns

Index(['name', 'cmc', 'B', 'C', 'G', 'N', 'R', 'U', 'W', 'legendary',
       'types_embedding', 'sub_type_embedding', 'keywords_embedding', 'power',
       'toughness', 'reserved', 'rarity_common', 'rarity_mythic',
       'rarity_rare', 'rarity_special', 'rarity_uncommon', 'released_at',
       'booster', 'oracle_text', 'usd'],
      dtype='object')

In [84]:
df = df[['name', 'cmc', 'B', 'C', 'G', 'N', 'R', 'U', 'W', 'legendary',
       'types_embedding', 'sub_type_embedding', 'keywords_embedding', 'power',
       'toughness', 'reserved', 'rarity_common', 'rarity_mythic',
       'rarity_rare', 'rarity_special', 'rarity_uncommon', 'released_at',
       'booster','oracle_text', 'usd']]
df.head()

,name,cmc,B,C,G,...,rarity_uncommon,released_at,booster,oracle_text,usd
0,+2 Mace,2.0,0,0,0,...,0.0,2021-07-23,1,Equipped creature gets +2/+2. \nEquip 3,0.02
1,Aarakocra Sneak,4.0,0,0,0,...,0.0,2022-06-10,1,"Flying\nWhen Aarakocra Sneak enters the battlefield, you take the initiative.",0.08
2,Abaddon the Despoiler,5.0,1,0,0,...,0.0,2022-10-07,0,"Trample\nMark of Chaos Ascendant — During your turn, spells you cast from your hand with mana value X or less have cascade, where X is the total amount of life your opponents have lost this turn.",2.88
3,Abandoned Outpost,0.0,0,0,0,...,0.0,2001-10-01,1,"Abandoned Outpost enters the battlefield tapped. \nTap: Add White . \nTap, Sacrifice Abandoned Outpost: Add one mana of any color.",0.14
4,Abandoned Sarcophagus,3.0,0,1,0,...,0.0,2020-04-17,0,"You may cast spells that have a cycling ability from your graveyard. \nIf a card that has a cycling ability would be put into your graveyard from anywhere and it wasn't cycled, exile it instead.",0.09


In [85]:
df['oracle_text']=df['oracle_text'].str.replace('\n', ' ')
df.head()

,name,cmc,B,C,G,...,rarity_uncommon,released_at,booster,oracle_text,usd
0,+2 Mace,2.0,0,0,0,...,0.0,2021-07-23,1,Equipped creature gets +2/+2. Equip 3,0.02
1,Aarakocra Sneak,4.0,0,0,0,...,0.0,2022-06-10,1,"Flying When Aarakocra Sneak enters the battlefield, you take the initiative.",0.08
2,Abaddon the Despoiler,5.0,1,0,0,...,0.0,2022-10-07,0,"Trample Mark of Chaos Ascendant — During your turn, spells you cast from your hand with mana value X or less have cascade, where X is the total amount of life your opponents have lost this turn.",2.88
3,Abandoned Outpost,0.0,0,0,0,...,0.0,2001-10-01,1,"Abandoned Outpost enters the battlefield tapped. Tap: Add White . Tap, Sacrifice Abandoned Outpost: Add one mana of any color.",0.14
4,Abandoned Sarcophagus,3.0,0,1,0,...,0.0,2020-04-17,0,"You may cast spells that have a cycling ability from your graveyard. If a card that has a cycling ability would be put into your graveyard from anywhere and it wasn't cycled, exile it instead.",0.09


In [86]:
from transformers import DistilBertTokenizer, DistilBertModel

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')


def get_embeddings(sentence):
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].tolist()

/Users/louishagenbucher/Documents/GitHub/mtg_nlp_price_prediction/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [87]:
df['text_embeddings'] = df['oracle_text'].apply(get_embeddings)
df['text_embeddings']

0             [[-0.43040189146995544, -0.4970492124557495, 0.08531604707241058, -0.05800575762987137, -0.12929967045783997, -0.19761843979358673, 0.3016900420188904, 0.16926373541355133, -0.1278557926416397, -0.169758141040802, -0.3836471736431122, -0.028914984315633774, 0.024290552362799644, 0.03714811056852341, -0.013013919815421104, 0.2381449043750763, 0.029161468148231506, 0.21373620629310608, -0.024507418274879456, -0.007280466612428427, 0.2505044639110565, -0.1894274652004242, 0.1284918189048767, -0.21616598963737488, 0.0388571135699749, 0.010285962373018265, 0.205935537815094, 0.18301716446876526, -0.0048970431089401245, 0.1799987554550171, -0.21212758123874664, 0.01808515004813671, -0.27451983094215393, -0.3894905745983124, -0.065420962870121, -0.21215471625328064, 0.16277581453323364, -0.07513546198606491, 0.14316245913505554, -0.007883019745349884, -0.010989144444465637, -0.07166361808776855, 0.1620766669511795, 0.317140132188797, 0.05884219706058502, -0.31310996413230896, -2

In [88]:
df.head()

,name,cmc,B,C,G,...,released_at,booster,oracle_text,usd,text_embeddings
0,+2 Mace,2.0,0,0,0,...,2021-07-23,1,Equipped creature gets +2/+2. Equip 3,0.02,"[[-0.43040189146995544, -0.4970492124557495, 0.08531604707241058, -0.05800575762987137, -0.12929967045783997, -0.19761843979358673, 0.3016900420188904, 0.16926373541355133, -0.1278557926416397, -0.169758141040802, -0.3836471736431122, -0.028914984315633774, 0.024290552362799644, 0.03714811056852341, -0.013013919815421104, 0.2381449043750763, 0.029161468148231506, 0.21373620629310608, -0.024507418274879456, -0.007280466612428427, 0.2505044639110565, -0.1894274652004242, 0.1284918189048767, -0.21616598963737488, 0.0388571135699749, 0.010285962373018265, 0.205935537815094, 0.18301716446876526, -0.0048970431089401245, 0.1799987554550171, -0.21212758123874664, 0.01808515004813671, -0.27451983094215393, -0.3894905745983124, -0.065420962870121, -0.21215471625328064, 0.16277581453323364, -0.07513546198606491, 0.14316245913505554, -0.007883019745349884, -0.010989144444465637, -0.07166361808776855, 0.1620766669511795, 0.317140132188797, 0.05884219706058502, -0.31310996413230896, -2.726865768432617, -0.09945127367973328, -0.25525805354118347, -0.2629059851169586, 0.06047394871711731, 0.02610255777835846, -0.11308598518371582, 0.4463523328304291, 0.24480299651622772, 0.3048168420791626, 0.038742393255233765, 0.2550275921821594, 0.28374427556991577, 0.25220543146133423, 0.05502738058567047, -0.027305983006954193, 0.02785281091928482, 0.09771665185689926, -0.050992321223020554, 0.2513068616390228, -0.1786777675151825, 0.4779109060764313, -0.3508955240249634, 0.4404112994670868, -0.4090628921985626, -0.32307514548301697, 0.41444674134254456, -0.02827616222202778, -0.12277620285749435, -0.22625502943992615, -0.34932613372802734, 0.1601104736328125, 0.10544411838054657, 0.061762966215610504, 0.3269675076007843, 0.22701267898082733, 0.5158734917640686, -0.020157497376203537, 0.44176071882247925, 0.19049328565597534, -0.03429331257939339, -0.22564317286014557, -0.16779553890228271, 0.6740438938140869, 0.035297639667987823, -0.39886966347694397, -0.10614708065986633, 0.5049126148223877, 0.3899739384651184, 0.03839697688817978, -0.3710818886756897, -0.07131153345108032, 0.2888955771923065, 0.21253374218940735, ...]]"
1,Aarakocra Sneak,4.0,0,0,0,...,2022-06-10,1,"Flying When Aarakocra Sneak enters the battlefield, you take the initiative.",0.08,"[[-0.16684836149215698, -0.23930270969867706, 0.03308930993080139, -0.07403352111577988, -0.14197759330272675, -0.26376381516456604, 0.19890525937080383, 0.13508351147174835, -0.057062599807977676, -0.4413888156414032, -0.13143199682235718, -0.023327255621552467, 0.15663345158100128, 0.17876207828521729, 0.08881733566522598, 0.19994071125984192, -0.17115724086761475, 0.1588028371334076, 0.07020659744739532, 0.07624676078557968, 0.03768850117921829, -0.2320052683353424, 0.28972262144088745, 0.11116282641887665, 0.09850765764713287, -0.1276589184999466, 0.2346213161945343, 0.17773142457008362, 0.073888398706913, -0.14035484194755554, -0.12160367518663406, 0.06699325144290924, -0.21209093928337097, -0.31720808148384094, 0.14610224962234497, 0.10261629521846771, 0.09934668242931366, 0.15840646624565125, 0.12436152994632721, 0.06545747816562653, -0.11312682926654816, 0.1351132094860077, 0.0688817948102951, 0.03316884487867355, 0.059165626764297485, -0.37473559379577637, -2.369394540786743, -0.2607436776161194, -0.031222309917211533, -0.3935493528842926, 0.07753954827785492, 0.14525803923606873, 0.023659545928239822, 0.11995726078748703, 0.3158957362174988, 0.4629892408847809, -0.39027029275894165, 0.5791729688644409, -0.04567687213420868, 0.07891136407852173, 0.03638871759176254, 0.022532425820827484, -0.13643278181552887, 0.11809278279542923, -0.056235019117593765, 0.08487822115421295, 0.03617187216877937, 0.21448253095149994, -0.4072163999080658, 0.27075818181037903, -0.29167553782463074, -0.048455849289894104, 0.197397381067276, -0.23009374737739563, -0.14893

In [89]:
df.columns

Index(['name', 'cmc', 'B', 'C', 'G', 'N', 'R', 'U', 'W', 'legendary',
       'types_embedding', 'sub_type_embedding', 'keywords_embedding', 'power',
       'toughness', 'reserved', 'rarity_common', 'rarity_mythic',
       'rarity_rare', 'rarity_special', 'rarity_uncommon', 'released_at',
       'booster', 'oracle_text', 'usd', 'text_embeddings'],
      dtype='object')

In [90]:
df = df[['name', 'cmc', 'B', 'C', 'G', 'N', 'R', 'U', 'W', 'legendary',
       'types_embedding', 'sub_type_embedding', 'keywords_embedding', 'power',
       'toughness', 'reserved', 'rarity_common', 'rarity_mythic',
       'rarity_rare', 'rarity_special', 'rarity_uncommon', 'released_at',
       'booster', 'text_embeddings', 'usd']]
df.columns

Index(['name', 'cmc', 'B', 'C', 'G', 'N', 'R', 'U', 'W', 'legendary',
       'types_embedding', 'sub_type_embedding', 'keywords_embedding', 'power',
       'toughness', 'reserved', 'rarity_common', 'rarity_mythic',
       'rarity_rare', 'rarity_special', 'rarity_uncommon', 'released_at',
       'booster', 'text_embeddings', 'usd'],
      dtype='object')

In [91]:
df.head()

,name,cmc,B,C,G,...,rarity_uncommon,released_at,booster,text_embeddings,usd
0,+2 Mace,2.0,0,0,0,...,0.0,2021-07-23,1,"[[-0.43040189146995544, -0.4970492124557495, 0.08531604707241058, -0.05800575762987137, -0.12929967045783997, -0.19761843979358673, 0.3016900420188904, 0.16926373541355133, -0.1278557926416397, -0.169758141040802, -0.3836471736431122, -0.028914984315633774, 0.024290552362799644, 0.03714811056852341, -0.013013919815421104, 0.2381449043750763, 0.029161468148231506, 0.21373620629310608, -0.024507418274879456, -0.007280466612428427, 0.2505044639110565, -0.1894274652004242, 0.1284918189048767, -0.21616598963737488, 0.0388571135699749, 0.010285962373018265, 0.205935537815094, 0.18301716446876526, -0.0048970431089401245, 0.1799987554550171, -0.21212758123874664, 0.01808515004813671, -0.27451983094215393, -0.3894905745983124, -0.065420962870121, -0.21215471625328064, 0.16277581453323364, -0.07513546198606491, 0.14316245913505554, -0.007883019745349884, -0.010989144444465637, -0.07166361808776855, 0.1620766669511795, 0.317140132188797, 0.05884219706058502, -0.31310996413230896, -2.726865768432617, -0.09945127367973328, -0.25525805354118347, -0.2629059851169586, 0.06047394871711731, 0.02610255777835846, -0.11308598518371582, 0.4463523328304291, 0.24480299651622772, 0.3048168420791626, 0.038742393255233765, 0.2550275921821594, 0.28374427556991577, 0.25220543146133423, 0.05502738058567047, -0.027305983006954193, 0.02785281091928482, 0.09771665185689926, -0.050992321223020554, 0.2513068616390228, -0.1786777675151825, 0.4779109060764313, -0.3508955240249634, 0.4404112994670868, -0.4090628921985626, -0.32307514548301697, 0.41444674134254456, -0.02827616222202778, -0.12277620285749435, -0.22625502943992615, -0.34932613372802734, 0.1601104736328125, 0.10544411838054657, 0.061762966215610504, 0.3269675076007843, 0.22701267898082733, 0.5158734917640686, -0.020157497376203537, 0.44176071882247925, 0.19049328565597534, -0.03429331257939339, -0.22564317286014557, -0.16779553890228271, 0.6740438938140869, 0.035297639667987823, -0.39886966347694397, -0.10614708065986633, 0.5049126148223877, 0.3899739384651184, 0.03839697688817978, -0.3710818886756897, -0.07131153345108032, 0.2888955771923065, 0.21253374218940735, ...]]",0.02
1,Aarakocra Sneak,4.0,0,0,0,...,0.0,2022-06-10,1,"[[-0.16684836149215698, -0.23930270969867706, 0.03308930993080139, -0.07403352111577988, -0.14197759330272675, -0.26376381516456604, 0.19890525937080383, 0.13508351147174835, -0.057062599807977676, -0.4413888156414032, -0.13143199682235718, -0.023327255621552467, 0.15663345158100128, 0.17876207828521729, 0.08881733566522598, 0.19994071125984192, -0.17115724086761475, 0.1588028371334076, 0.07020659744739532, 0.07624676078557968, 0.03768850117921829, -0.2320052683353424, 0.28972262144088745, 0.11116282641887665, 0.09850765764713287, -0.1276589184999466, 0.2346213161945343, 0.17773142457008362, 0.073888398706913, -0.14035484194755554, -0.12160367518663406, 0.06699325144290924, -0.21209093928337097, -0.31720808148384094, 0.14610224962234497, 0.10261629521846771, 0.09934668242931366, 0.15840646624565125, 0.12436152994632721, 0.06545747816562653, -0.11312682926654816, 0.1351132094860077, 0.0688817948102951, 0.03316884487867355, 0.059165626764297485, -0.37473559379577637, -2.369394540786743, -0.2607436776161194, -0.031222309917211533, -0.3935493528842926, 0.07753954827785492, 0.14525803923606873, 0.023659545928239822, 0.11995726078748703, 0.3158957362174988, 0.4629892408847809, -0.39027029275894165, 0.5791729688644409, -0.04567687213420868, 0.07891136407852173, 0.03638871759176254, 0.022532425820827484, -0.13643278181552887, 0.11809278279542923, -0.056235019117593765, 0.08487822115421295, 0.03617187216877937, 0.21448253095149994, -0.4072163999080658, 0.27075818181037903, -0.29167553782463074, -0.048455849289894104, 0.197397381067276, -0.23009374737739563, -0.14893658459186554, -0.11193645000457764, -0.2122679501771927, 0.07849310338497162, -0.15088890492916107, 0.18620386